In [18]:
from lazylabel.text.all import *
from fastai2.text.all import *
from fastai2.tabular.all import *
from textblob import TextBlob

In [80]:
pd.options.display.max_colwidth = 9999

In [2]:
source = untar_data(URLs.IMDB_SAMPLE)

In [3]:
source.ls()

(#1) [Path('/home/lgvaz/.fastai/data/imdb_sample/texts.csv')]

In [4]:
df = pd.read_csv(source/'texts.csv')
df.head(1)

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False


In [5]:
df['original'] = df['text']

## Create labellers

In [6]:
ABSTAIN,POS,NEG = 'abstain','positive','negative'
vocab = CategoryMap([ABSTAIN,POS,NEG])

In [9]:
splits = ColSplitter('is_valid')(df)

In [13]:
cder,tkzer,nmzer = map(mk_transform, [ColReader('original'), Tokenizer.from_df('text'), Numericalize])
x_tls = TfmdLists(df, [cder,tkzer,nmzer], splits=splits)
show_at(x_tls, 0)

xxbos xxmaj un - xxunk - believable ! xxmaj meg xxmaj ryan does n't even look her usual xxunk lovable self in this , which normally makes me forgive her shallow xxunk acting xxunk . xxmaj hard to believe she was the producer on this dog . xxmaj plus xxmaj kevin xxmaj kline : what kind of suicide trip has his career been on ? xxmaj xxunk … xxmaj xxunk xxrep 3 ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . w xxrep 3 o xxrep 3 f !


In [15]:
y_tls = TfmdLists(df, [ColReader('label'), Categorize(vocab)], splits=splits)
show_at(y_tls, 0)

negative


In [16]:
dset = Datasets(tls=[x_tls, y_tls])

In [30]:
labeller = Labeller()

In [83]:
def keyword_match(ws, label):
    ws = L(ws)
    def _inner(x):
        for w in ws:
            if w in x: return label
        return ABSTAIN
    _inner.__name__ = f'kw_{ws[0]}'
    labeller.register_func(tkzer, _inner)

In [54]:
@labeller(cder, TextBlob)
def tb_polarity(x, tb):
    if tb.polarity > .2:  return POS
    if tb.polarity < .1: return NEG
    return ABSTAIN

In [115]:
labeller.reset()

In [86]:
keyword_match(['hate', 'hated'], NEG)

In [107]:
keyword_match(['loved'], POS)

In [116]:
keyword_match(['heck'], POS)

In [117]:
tasks = tasks_labels(labeller, x_tls, vocab)
dset = Datasets(tls=[tasks, y_tls])
dls = dset.dataloaders()
labeller.summary(dls.train)

,Coverage,Polarity,Accuracy,Correct,Incorrect
kw_heck,1.17% (9),1,0.444444,4,5


In [109]:
idxs = labeller.find(dls.train, vocab, ['kw_heck'], [POS])

In [113]:
df[['text', 'label']].iloc[idxs].sample(5)

,text,label
200,"I loved the first ""Azumi"" movie. I've seen Ms. Ueto in a variety of her TV appearances and I've seen my fair share of samurai and ninja flicks. I have to say that this movie was much weaker than I'd expected.<br /><br />Given the movie's cast and set up in ""Azumi"", they should have been able to do a much better job with this movie, but instead it was slow, plodding in parts, and sprinkled with very poor, unconvincing, and wooden acting.<br /><br />When they bothered to reference the first movie, they did so in a manner that was pretty loose and weak. In ""Azumi"", the title character is the best of a group of superior killers. In ""Azumi 2"" she seems somehow diminished and less-impressive.<br /><br />That's not to say it was a total loss. There were a few decent fight scenes and some over-the-top characters. Unfortunately, the movie suffers overall from the simple fact that Shusuke Kaneko and Yoshiaki Kawajiri are not Ryuhei Kitamura and Isao Kiriyama. The latter two truly captured the ""manga"" feel in their screenplay whereas the former never quite ""got it.""",negative
398,"A friend once asked me to read a screenplay of his that had been optioned by a movie studio. To say it was one of the most inept and insipid scripts I'd ever read would be a bold understatement. Yet I never told him this. Why? Because in a world where films like ""While She Was Out"" can be green-lighted and attract an Oscar- winning star like Kim Basinger, a screenplay lacking in character, content and common sense is no guarantee that it won't sell.<br /><br />As so many other reviewers have pointed out, ""While She Was Out"" is a dreadfully under-written Woman-in-Peril film that has abused housewife Basinger hunted by four unlikely hoods on Christmas Eve. Every gripe is legitimate, from the weak dialog and bad acting to the jaw-dropping lapses of logic, but Basinger is such an interesting actress and the premise is not without promise. Here are a couple of things that struck me:<br /><br />1) I don't care how much we are supposed to think her husband is a jerk, the house IS a mess with toys. Since when did it become child abuse to make kids pick up after themselves?<br /><br />2) Racially diverse gangs are rare everywhere except Hollywood, where they are usually the only racially balanced groups on screen.<br /><br />3) Sure the film is stupid. But so are the countless ""thrillers"" I've sat through where the women are portrayed as wailing, helpless victims of male sadism. Stupid or not, I found it refreshing to see a woman getting the best of her tormentors.<br /><br />4) I LOVED the ending! <br /><br />5) Though an earlier reviewer coined this phrase, I really DO think this film should be retitled ""The Red Toolbox of Doom.""",negative
600,"15 years ago, Power Rangers was one of the shows that swept the nations, especially that of the youth. When I was a 3-year-old kid living in the Philippines, I would watch this show every Friday on ABS-CBN (Channel 2 over there). When the movie came out in 1995, it was all shock and awe to me when I watched it with my dad and 2 uncles. My grandparents even sent me a ""Balikbayan"" box full of Power Rangers stuff (including the easy-to-build Megazord figure and the stuffed toys of the Rangers). I even played the video games, and loved only one of them (""MMPR: Fighting Edition""), which was REALLY sad. Anyway, it was the show that set a stepping stone on my interest on robot series (especially anime mech series like ""Gundam"") Now that I'm 18, I'd like to think this show is pretty cheesy to me now. To prove this, I took a trip down memory lane by buying a 2-set DVD collection with ""MMPR: The Movie"" and ""Turbo: A Power Rangers Movie"" and watched both of them. I was like, ""Ugh. How awful. Just another junk in the attic for me."" For all of you people of my generation, this is NOT a show that you'd want to watch over and over again. I'm not saying to leave it behind for the rest of your li

In [73]:
df.iloc[idxs].sample(5)

,label,text,is_valid,original
395,negative,"...okay, maybe not all of it. Lured by the false promise of bikini-clad women on the movie's cover...but the HORROR...THE HORROR... ...whatever you do, do NOT watch this movie. Gouge out your eyes, repeatedly bash your skull in...do what it takes. Never again--never forget!<br /><br />",False,"...okay, maybe not all of it. Lured by the false promise of bikini-clad women on the movie's cover...but the HORROR...THE HORROR... ...whatever you do, do NOT watch this movie. Gouge out your eyes, repeatedly bash your skull in...do what it takes. Never again--never forget!<br /><br />"
453,positive,"Silverlake Life, The view from here, is an absolutely stunning movie about AIDS as well as about a gay love relationship. Some images are indeed really hard to take, especially when one is gay or fears about AIDS, and probably for any sensitive person watching it. It's not easy to make a movie about such a terrible illness and its consequences about not only one, but two people's lives. This movie teaches how to care for each other in such hard times, but it never gets too morbid, it still shows life at any time, reminding you that outside of the theater or of your room, life goes on, what...",False,"Silverlake Life, The view from here, is an absolutely stunning movie about AIDS as well as about a gay love relationship. Some images are indeed really hard to take, especially when one is gay or fears about AIDS, and probably for any sensitive person watching it. It's not easy to make a movie about such a terrible illness and its consequences about not only one, but two people's lives. This movie teaches how to care for each other in such hard times, but it never gets too morbid, it still shows life at any time, reminding you that outside of the theater or of your room, life goes on, what..."
222,negative,"May contain spoilers.<br /><br />I say that, but anyone savvy enough to be reading this can probably figure out every plot turn right from the start.<br /><br />This is not a movie that I liked. I didn't hate it in the way of some movies that insult your intelligence, but it all felt too predictable on its trudge to the requisite happy ending. There were funny bits along the way to be sure, but few were original. At least it didn't go for the gutter.<br /><br />Christina Applegate looks fresh, and Ben Affleck works hard. Their scenes together are actually the only redeeming feature. Everyo...",False,"May contain spoilers.<br /><br />I say that, but anyone savvy enough to be reading this can probably figure out every plot turn right from the start.<br /><br />This is not a movie that I liked. I didn't hate it in the way of some movies that insult your intelligence, but it all felt too predictable on its trudge to the requisite happy ending. There were funny bits along the way to be sure, but few were original. At least it didn't go for the gutter.<br /><br />Christina Applegate looks fresh, and Ben Affleck works hard. Their scenes together are actually the only redeeming feature. Everyo..."
384,positive,"Following the brilliant ""Goyôkiba"" (aka. ""Hanzo The Razor - Sword Of Justice"", 1972) and its excellent (and even sleazier) sequel ""Goyôkiba: Kamisori Hanzô jigoku zeme"" (aka. ""Razor 2: The Snare"", 1973), this ""Goyôkiba: Oni no Hanzô yawahada koban"" aka. ""Razor 3: Who's Got The Gold"" is the third, and sadly final installment to the awesome saga about the incorruptible Samurai-constable Hanzo 'The Razor' Ittami (brilliantly played by the great Shintarô Katsu), who fights corruption with his fighting expertise as well as his enormous sexual powers. As a big fan of 70s exploitation cinema made...",False,"Following the brilliant ""Goyôkiba"" (aka. ""Hanzo The Razor - Sword Of Justice"", 1972) and its excellent (and even sleazier) sequel ""Goyôkiba: Kamisori Hanzô jigoku zeme"" (aka. ""Razor 2: The Snare"", 1973), this ""Goyôkiba: Oni no Hanzô yawahada koban"" aka. ""Razor 3: Who's Got The Gold

In [65]:
df.sample(5)

,label,text,is_valid,original
118,negative,"Anita and Me seems to be little more than an excuse for Meera Syal, the author of the novel and screenplay, to air her prejudices, grievances and general antipathy towards the English. The general sentiment of Indian superiority over the English in this film is foul.<br /><br />The English people in this film are portrayed as overweight, violent, foul-mouthed, promiscuous, engaging in child neglect, stupid, uneducated, racist, ugly, eating poor food, and dim-witted -- tellingly, only by turning to Indian culture can the local priest be ""redeemed"" at the end of the film.<br /><br />By contr...",False,"Anita and Me seems to be little more than an excuse for Meera Syal, the author of the novel and screenplay, to air her prejudices, grievances and general antipathy towards the English. The general sentiment of Indian superiority over the English in this film is foul.<br /><br />The English people in this film are portrayed as overweight, violent, foul-mouthed, promiscuous, engaging in child neglect, stupid, uneducated, racist, ugly, eating poor food, and dim-witted -- tellingly, only by turning to Indian culture can the local priest be ""redeemed"" at the end of the film.<br /><br />By contr..."
654,positive,"Sherman, set the wayback machine for... 1986. The United States was just climbing out of its worst postwar recession, while Japan was enjoying an unprecedented industrial boom. Manufacturing industries were still a significant part of the US economy, and factory workers were a good example of the ""average American"". The word ""downsizing"" hadn't entered the general vocabulary yet, but everyone knew the phenomenon. Bruce could be heard on the radio singing, ""Foreman says these jobs are going, boy, and they ain't coming back to your hometown."" Chrysler had just been bailed out by Uncle Sam. B...",False,"Sherman, set the wayback machine for... 1986. The United States was just climbing out of its worst postwar recession, while Japan was enjoying an unprecedented industrial boom. Manufacturing industries were still a significant part of the US economy, and factory workers were a good example of the ""average American"". The word ""downsizing"" hadn't entered the general vocabulary yet, but everyone knew the phenomenon. Bruce could be heard on the radio singing, ""Foreman says these jobs are going, boy, and they ain't coming back to your hometown."" Chrysler had just been bailed out by Uncle Sam. B..."
282,positive,"If in the 90's you're adapting a book written in the 50's, set the bloody thing in the 50's and not the '90's. See, 40 year old mores and values tend not to play as well, or ring as true, that far down the road. It's a simple rule that Hollywood habitually keeps violating. And that's the problem with this film. It should have been set in the era it was written in. You'd think that would be a no-brainer, but nooo. I'd elaborate, but bmacv's comment spells it out quite well. I'll limit my commentary to Rachel Ward. She looks like she dieted her ass completely out of existence for this role. ...",False,"If in the 90's you're adapting a book written in the 50's, set the bloody thing in the 50's and not the '90's. See, 40 year old mores and values tend not to play as well, or ring as true, that far down the road. It's a simple rule that Hollywood habitually keeps violating. And that's the problem with this film. It should have been set in the era it was written in. You'd think that would be a no-brainer, but nooo. I'd elaborate, but bmacv's comment spells it out quite well. I'll limit my commentary to Rachel Ward. She looks like she dieted her ass completely out of existence for this role. ..."
253,positive,"Movies like this one, and C.R.A.Z.Y., make me very sad for American films with a gay subject matter. With the exception of Parting Glances and Brokeback Mountain, there are few other notable American films with the kind of depth and sincerity as this movie, The Bubble. T